In [1]:
suppressPackageStartupMessages(library(rstanarm))
suppressPackageStartupMessages(library(ggformula))
library(tibble)
suppressPackageStartupMessages(library(glue))
suppressPackageStartupMessages(library(dplyr))
suppressPackageStartupMessages(library(modelr))
library(stringr)

In [2]:
# Set the maximum number of columns and rows to display
options(repr.matrix.max.cols=150, repr.matrix.max.rows=200)
# Set the default plot size
options(repr.plot.width=18, repr.plot.height=12)

In [3]:
download_if_missing <- function(filename, url) {
    if (!file.exists(filename)) {
        dir.create(dirname(filename), showWarnings=FALSE, recursive=TRUE)
        download.file(url, destfile = filename, method="curl")
    }
}

# Fitting logistic regression to data

The folder [NES](https://github.com/avehtari/ROS-Examples/tree/master/NES/) contains sthe survey data of presidential preference and income for the 1992 election analyzed in Section 13.1, along with other variables including sex, ethnicity, education, party identification, and political ideology.

In [4]:
filename <- 'data/nes/nes.txt'

download_if_missing(filename, 'https://raw.githubusercontent.com/avehtari/ROS-Examples/master/NES/data/nes.txt')

nes <- read.table(filename)

In [7]:
print(nrow(nes))
nes %>% head()

[1] 34908


,year,resid,weight1,weight2,weight3,age,gender,race,educ1,urban,region,income,occup1,union,religion,educ2,educ3,martial_status,occup2,icpsr_cty,fips_cty,partyid7,partyid3,partyid3_b,str_partyid,father_party,mother_party,dlikes,rlikes,dem_therm,rep_therm,regis,vote,regisvote,presvote,presvote_2party,presvote_intent,ideo_feel,ideo7,ideo,cd,state,inter_pre,inter_post,black,female,age_sq,rep_presvote,rep_pres_intent,south,real_ideo,presapprov,perfin1,perfin2,perfin,presadm,age_10,age_sq_10,newfathe,newmoth,parent_party,white,year_new,income_new,age_new,vote.1,age_discrete,race_adj,dvote,rvote
,<int>,<int>,<dbl>,<dbl>,<dbl>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<dbl>,<dbl>,<int>,<int>,<int>,<int>,<int>,<int>,<dbl>,<int>,<int>,<dbl>,<int>,<int>
536,1952,1,1,1,1,25,2,1,2,2,1,4,2,1,1,3,3,1,2,NA,NA,6,3,3,3,3,3,0,1,NA,NA,2,2,3,2,2,2,NA,NA,NA,NA,13,50,NA,0,1,625,1,1,0,NA,NA,NA,NA,NA,-1,2.5,6.250000,1,1,2,1,1,1,-2.052455,1,1,1,0,1
537,1952,2,1,1,1,33,2,1,1,2,1,4,6,1,1,1,1,1,6,NA,NA,5,3,3,2,2,2,-1,3,NA,NA,2,2,3,1,1,2,NA,NA,NA,NA,13,50,NA,0,1,1089,0,1,0,NA,NA,NA,NA,NA,-1,3.3,10.889999,0,0,0,1,1,1,-1.252455,1,2,1,1,0
538,1952,3,1,1,1,26,2,1,2,2,1,3,6,2,2,3,3,1,6,NA,NA,4,2,2,1,1,1,0,5,NA,NA,2,2,3,2,2,2,NA,NA,NA,NA,13,50,NA,0,1,676,1,1,0,NA,NA,NA,NA,NA,-1,2.6,6.759999,-1,-1,-2,1,1,0,-1.952455,1,1,1,0,1
539,1952,4,1,1,1,63,1,1,2,2,1,3,3,1,1,2,2,1,3,NA,NA,7,3,3,4,1,NA,-1,3,NA,NA,1,2,3,2,2,2,NA,NA,NA,NA,13,50,NA,0,0,3969,1,1,0,NA,NA,NA,NA,NA,-1,6.3,39.690002,-1,NA,NA,1,1,0,1.747545,1,3,1,0,1
540,1952,5,1,1,1,66,2,1,2,2,2,1,6,2,1,4,4,1,6,NA,NA,7,3,3,4,1,1,-2,0,NA,NA,2,2,3,2,2,2,NA,NA,NA,NA,24,49,NA,0,1,4356,1,1,0,NA,NA,NA,NA,NA,-1,6.6,43.559998,-1,-1,-2,1,1,-2,2.047545,1,4,1,0,1
541,1952,6,1,1,1,48,2,1,2,2,2,4,6,1,1,2,2,1,6,NA,NA,3,1,1,2,1,1,0,4,NA,NA,2,2,3,2,2,2,NA,NA,NA,NA,24,49,NA,0,1,2304,1,1,0,NA,NA,NA,NA,NA,-1,4.8,23.040001,-1,-1,-2,1,1,1,0.247545,1,3,1,0,1


## Fit logistic regression models

Fit a logistic regression predicting support for Bush given all these inputs.
Consider how to include these as regression predictors and also consider possible interactions.

## Evaluate and compare models

Evaluate and compare the different models you have fit.

## Variable importance

For the chosen model, discuss and compare the importance of each input variable in the prediction.

# Sketching the logistic curve

Sketch the following logistic regression curves with pen on paper:

a) $ {\rm Pr}(y = 1) = {\rm logit}^{-1}(x) $

b) $ {\rm Pr}(y = 1) = {\rm logit}^{-1}(2+x) $

c) $ {\rm Pr}(y = 1) = {\rm logit}^{-1}(2x) $

d) $ {\rm Pr}(y = 1) = {\rm logit}^{-1}(2+2x) $

e) $ {\rm Pr}(y = 1) = {\rm logit}^{-1}(-2x) $

# Understanding logistic regression coefficients

In Chapter 7 we fit a model predicting incumbent party's two-party vote percentage given economic growth: ${\rm vote} = 46.2 + 3.1 \times {\rm growth} + {\rm error} $, where `growth` ranges from -0.5 to 5.6 in the data, and errors are approximately normally distributed with mean 0 and standard deviation 3.8.
Suppose instead we were to fit a logitic regression, $ {\rm Pr}({\rm vote} > 50) = {\rm logit}^{-1}(a + b \times {\rm growth}) $. Approximately what are the estimates of $(a, b)$.

Figure this out in four steps:

<ol style="list-style-type:lower-roman;">
     
<li> Use the fitted linear regression model to estimate Pr(vote > 50) for different values of `growth`
    </li>
<li>Plot these probabilities and draw a logistic curve through them
    </li>
<li> Use the divide-by-4 rule to estimate the slope of the logistic regression model
    </li>
<li> Use the point where the probability goes through 0.5 to deduce the intercept.
    </li>
    </ol>
Do all this using the above information, without downloading the data and fitting the model.

### Checking with the data

In [20]:
filename <- "./data/ElectionsEconomy/hibbs.dat"

download_if_missing(filename,
                    'https://raw.githubusercontent.com/avehtari/ROS-Examples/master/ElectionsEconomy/data/hibbs.dat')
hibbs <- read.table(filename, header=TRUE)

# Logistic regression with two predictors

The following logistic regression has been fit:

```
            Median    MAD_SD
(Intercept)  -1.9      0.6
x             0.7      0.8
z             0.7      0.5
```

Here, $x$ is a continuous predictor ranging from 0 to 10, and $z$ is a binary predictor taking on the values 0 and 1.
Display the fitted model as two curves on a graph of ${\rm Pr}(y=1)$ vs $x$.

# Intepreting logistic regression coefficients

Here is a fitted model from the Bangladesh analysis predicting whether a person with high-arsenic drinking water will switch wells, given the arsenic level in their existing well and the disance to the nearest safe well:

```
stan_glm(formula = switch ~ dist100 + arsenic, family=binomial(link="logit"), data=wells)

            Median    MAD_SD
(Intercept)  0.00      0.08
dist100     -0.90      0.10
arsenic      0.46      0.04
```

Compare two people who live the same distance from the nearest well bus whose arsenic levels differ, with one person having an arsenic level of 0.5 and the other person having a level of 1.0.
You will estimate how much more likely this second person is to switch wells.
Give an approximate estimate, standard error, 50% interval, 95% interval, using two different methods.

## Divide-by-4

Use the divide-by-4 rule, based on the information from this regression output.

## Predictive simulation

Use predictive simulation from the fitted model in R, under the assumption that these two people each live 50 meters from the nearest well.

# Interpreting logistic regression coefficient uncertainties

In Section 14.2, there were two models, `fit_4` and `fit_5`, with distance and arsenic levels as predictors along with an interaction term.
The model `fit_5` differed by using centred predictors.
Compare the reported uncertainty estimates (mad sd) for the coefficients, and use for example the `mcmc_pairs` function in the `bayesplot` package to examine the pairwise joint posterior distributions.
Explain why the mad sd values are different for `fit_4` and `fit_5`.

In [26]:
filename <- "./data/Arsenic/wells.csv"

download_if_missing(filename,
                    'https://raw.githubusercontent.com/avehtari/ROS-Examples/master/Arsenic/data/wells.csv')

wells <- read.csv(filename)

In [29]:
wells <- wells %>% 
mutate(dist100 = dist/100,
       c_dist100 = dist100 - mean(dist100),
       c_arsenic = arsenic - mean(arsenic))

In [30]:
(fit4 <- stan_glm(switch ~ dist100 + arsenic + dist100:arsenic,
                family=binomial(link="logit"),
                data=wells,
                refresh=0))

stan_glm
 family:       binomial [logit]
 formula:      switch ~ dist100 + arsenic + dist100:arsenic
 observations: 3020
 predictors:   4
------
                Median MAD_SD
(Intercept)     -0.1    0.1  
dist100         -0.6    0.2  
arsenic          0.6    0.1  
dist100:arsenic -0.2    0.1  

------
* For help interpreting the printed output see ?print.stanreg
* For info on the priors used see ?prior_summary.stanreg

In [31]:
(fit5 <- stan_glm(switch ~ c_dist100 + c_arsenic + c_dist100:c_arsenic,
                family=binomial(link="logit"),
                data=wells,
                refresh=0))

stan_glm
 family:       binomial [logit]
 formula:      switch ~ c_dist100 + c_arsenic + c_dist100:c_arsenic
 observations: 3020
 predictors:   4
------
                    Median MAD_SD
(Intercept)          0.4    0.0  
c_dist100           -0.9    0.1  
c_arsenic            0.5    0.0  
c_dist100:c_arsenic -0.2    0.1  

------
* For help interpreting the printed output see ?print.stanreg
* For info on the priors used see ?prior_summary.stanreg

# Graphing a fitted logistic regression

We downloaded data with weight (in pounds) and age (in years) from a random sample of American adults.
We then defined a new variable:

```
heavy <- weight > 200
```

and fit a logistic regression, predicting `heavy` from `height` (in inches).

```
stan_glm(formula = heavy ~ height, family=binomial(link="logit"), data=health)

            Median    MAD_SD
(Intercept) -21.51     1.60
height        0.28     0.02
```

## Graph the curve

Graph the logistic regression curve (the probability that someone is heavy) over the approximate range of the data.
Be clear where the line goes through the 50% probability piont.

## Fill in the blank

Near the 50% point, comparing two people who differ by one inch in height, you'll expect a difference of **__** in the probability of being heavy.

# Linear transformations

In the regression from the previous exercise, suppose you replace height in inches by height in centimeters.
What would then be the intercept and slope?

# The algebra of logistic regression with one predictor

You are interested in how well the combined earnings of the parents in a child's family predicts high school graduation.
You are told that the probability a child graduates from high school is 27% for children whose parents earn no income and is 88% for clidren whose parents earn \\$60&thinsp;000.
Determine the logistic regression model that is consistent with this information.
For simplicity, you may want to assume that income is measured in units of \\$10&thinsp;000.

# Expressing a comparison of proportions as a logistic regression

A randomized experiment is performed within a survey, and 1000 people are contacted.
Half the people contactes are promised a \\$5 incentive to participate, and half are not promised and incentive.
The result is a 50% response rate among the treated group and 40% response rate among the control group.

## Fitting Logistic Regression

Set up these results as data in R.
From these data, fit a logistic regression of response on the treatment indicator.

## Comparing with Statistical Inference

Compare to the results from Exercise 4.1.

# Building a logistic regression model

The folder [Rodents](https://github.com/avehtari/ROS-Examples/tree/master/Rodents) contains data on rodents in a sample of New York City apartments.

In [17]:
filename <- 'data/Rodents/rodents.dat'

download_if_missing(filename, 'https://raw.githubusercontent.com/avehtari/ROS-Examples/master/Rodents/rodents.dat')

rodents <- read.table(filename)

# Fake-data simulation to evaluate a statistical procedure

When can we get away with fitting linear regression to binary data?
You will explore this question by simulating data from a logistic regression, then fitten a lienar regression, then looping this procedure to compute the coverate of the estimates.

## Simulation

You will be simulating indepentent binaty data, $y_i, i=1,\ldots,n$, from the model, ${\rm Pr}(y_i = 1) = {\rm logit}^{-1}(a + bx_i + \theta z_i$), where the $x_i$'s are drawn uniformly from the range (0, 100) and the $z_i$'s are randomly set to 0 or 1.
The "cover story" here is that $y$ represents passing or failing an exam, $x$ is the score on a pre-test, and $z$ is a treatment.

To do this simulation, you will need to set true values of $a$, $b$, and $\theta$. Choose $a$ and $b$ so that 60% of the students in the control group will pass the exam, with the probability of passing being 80% for students in the control group who scored 100 on the midterm.

Choose $\theta$ so that the average probability of passing increases by 10 percentage points under the treatment.

Report your values fo $a, b, \theta$ and explain your reasoning (including simulation code).
It's not enough just to guess.

## Fitting a linear regression

Simulate $n=50$ data points from you model, and then fit a linear regression of $y$ on $x$ and $z$.
Look at the estimate and standard error for the coefficient of $z$.
Does the true average treatment effect fall inside this interval?

## Coverage

Repeat you simulation 10&thinsp;000 times.
Compute the coverage of the normal-theory 50% and 95% intervals (that is, the estimates of ±0.67 and 1.96 standard errors).